References: <br>
https://github.com/bentrevett/pytorch-seq2seq <br>
https://pytorch.org/tutorials/beginner/data_loading_tutorial.html<br>
Source Directory: https://www.statmt.org/europarl/ <br>
Data directory: https://drive.google.com/drive/folders/1Wwz0WPKWuc1RimMLze9klzAdRS-nGNw1?usp=sharing <br>
Please have all the files in the same folder for the code to execute

Importing the libraries: We use Pytorch to train our models

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchtext
from torchtext.data import TabularDataset
from torchtext.data import Field, BucketIterator

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from spacy.lang.en import English
from spacy.lang.fr import French
from spacy.lang.es import Spanish
import numpy as np
import torch.nn.functional as F
import random
import math
import time
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu

Randomize the parameters for pytorch

In [0]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [0]:
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cuda'

Loading the tokenizer and tokenizing the sentences <br>
Creating field values for the source, target1, target2 <br>
target1 represents spanish <br>
target2 represents French

In [0]:
spacy_en = English()
spacy_es = Spanish()
spacy_fr = French()

In [0]:
def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

def tokenize_es(text):
    return [tok.text for tok in spacy_es.tokenizer(text)]

def tokenize_fr(text):
    return [tok.text for tok in spacy_fr.tokenizer(text)]

In [0]:
SRC = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True, 
            batch_first = True)

TRG1 = Field(tokenize = tokenize_es, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True, 
            batch_first = True)

TRG2 = Field(tokenize = tokenize_fr, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True, 
            batch_first = True)

In [0]:
tv_datafields = [("English", SRC),
                 ("Spanish", TRG1),
                 ("French", TRG2)]
trn, vld, tst = TabularDataset.splits(
               path = "./",
               train='train_df.csv', validation="val_df.csv", test="test_df.csv",
               format='csv',
               skip_header=True,
               fields=tv_datafields)

Building the vocabulary and creating batches for the model. <br>
We consider a batch size of 20

In [0]:
SRC.build_vocab(trn, min_freq = 2)
TRG1.build_vocab(trn, min_freq = 2)
TRG2.build_vocab(trn, min_freq = 2)

In [0]:
BATCH_SIZE = 20
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (trn, vld, tst),
     sort_key=lambda x: len(x.English),
     sort_within_batch=False,
     batch_size = BATCH_SIZE,
     device = device, 
     repeat=False)

Loading the pretrained GLOVE word embedding model and converting our vocab to 50 dimension weights

In [0]:
word_embeddings = {}
f = open('glove.6B.50d.txt', encoding='utf-8')

In [0]:
for line in f:
  values = line.split()
  word = values[0]
  coefs = np.asarray(values[1:], dtype='float32')
  word_embeddings[word] = coefs

In [0]:
INPUT_DIM = len(SRC.vocab) 
weight_encoder = np.zeros((INPUT_DIM, 50))
words_found = 0

for i, word in enumerate(SRC.vocab.itos):
    try: 
        weight_encoder[i] = word_embeddings[word]
        words_found += 1
    except KeyError:
        weight_encoder[i] = word_embeddings['unk']

INPUT_DIM_DEC1 = len(TRG1.vocab) 
weight_decoder1 = np.zeros((INPUT_DIM_DEC1, 50))
words_found = 0

for i, word in enumerate(TRG1.vocab.itos):
    try: 
        weight_decoder1[i] = word_embeddings[word]
        words_found += 1
    except KeyError:
        weight_decoder1[i] = word_embeddings['unk']

INPUT_DIM_DEC2 = len(TRG2.vocab) 
weight_decoder2 = np.zeros((INPUT_DIM_DEC2, 50))
words_found = 0

for i, word in enumerate(TRG2.vocab.itos):
    try: 
        weight_decoder1[i] = word_embeddings[word]
        words_found += 1
    except KeyError:
        weight_decoder1[i] = word_embeddings['unk']

Design of the network module:<br>
1) Encoder: We initialize the embedding layer with pre-trained word embedding for source<br>
2) EncoderLayer <br>
3) AttentionLayer <br>
5) Decoder: We pass the weight matrix as an argument and initialize the embedding for target <br>
6) DecoderLayer<br>

In [0]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        #Pretrained Word Embedding
        emb_weights = torch.Tensor(weight_encoder)
        self.embedding.weight = nn.Parameter(emb_weights)
        self.embedding.weight.requires_grad = False

        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)
        
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        embedded = self.dropout(self.embedding(src))
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded)       
        #outputs = [src len, batch size, hid dim * num directions]
        #hidden = [n layers * num directions, batch size, hid dim]
        
        #hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        #outputs are always from the last layer
        
        #hidden [-2, :, : ] is the last of the forwards RNN 
        #hidden [-1, :, : ] is the last of the backwards RNN
        
        #initial decoder hidden is final hidden state of the forwards and backwards 
        #  encoder RNNs fed through a linear layer
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
        
        #outputs = [src len, batch size, enc hid dim * 2]
        #hidden = [batch size, dec hid dim]
        outputs.view(outputs.shape[1], outputs.shape[0], outputs.shape[2])
        return outputs, hidden

In [0]:
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias = False)
        
    def forward(self, hidden, encoder_outputs):
        
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        #repeat decoder hidden state src_len times
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #hidden = [batch size, src len, dec hid dim]
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
        
        #energy = [batch size, src len, dec hid dim]

        attention = self.v(energy).squeeze(2)
        
        #attention= [batch size, src len]
        
        return F.softmax(attention, dim=1)

In [0]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention, weight_matrix):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        emb_weights = torch.Tensor(weight_matrix)
        self.embedding.weight = nn.Parameter(emb_weights)
        self.embedding.weight.requires_grad = False

        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
        
        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs):
             
        #input = [batch size]
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
        
        a = self.attention(hidden, encoder_outputs)
                
        #a = [batch size, src len]
        
        a = a.unsqueeze(1)
        
        #a = [batch size, 1, src len]
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        weighted = torch.bmm(a, encoder_outputs)
        
        #weighted = [batch size, 1, enc hid dim * 2]
        
        weighted = weighted.permute(1, 0, 2)
        
        #weighted = [1, batch size, enc hid dim * 2]
        rnn_input = torch.cat((embedded, weighted), dim = 2)
        
        #rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]
            
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        
        #output = [seq len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]
        
        #seq len, n layers and n directions will always be 1 in this decoder, therefore:
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
        #this also means that output == hidden
        assert (output == hidden).all()
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden.squeeze(0)

Seq2Seq module <br>
Here we specify the sequence to sequence module flow. <br>
We also employ teacher forcing parameter. <br>
The encoded source goes into both the decoders, decoder1 and decoder2

In [0]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder1, decoder2, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder1 = decoder1
        self.decoder2 = decoder2
        self.device = device
        
    def forward(self, src, trg1, trg2, teacher_forcing_ratio = 0.5):
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
        
        batch_size = src.shape[1]
        trg_len1 = trg1.shape[0]
        trg_vocab_size1 = self.decoder1.output_dim
        trg_len2 = trg2.shape[0]
        trg_vocab_size2 = self.decoder2.output_dim

        #tensor to store decoder outputs
        outputs1 = torch.zeros(trg_len1, batch_size, trg_vocab_size1).to(self.device)
        outputs2 = torch.zeros(trg_len2, batch_size, trg_vocab_size2).to(self.device)
        
        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src)
        
        hidden1 = hidden
        hidden2 = hidden        
        #first input to the decoder is the <sos> tokens
        input1 = trg1[0,:]
        input2 = trg2[0,:]
        
        for t in range(1, trg_len1):
            
            #insert input token embedding, previous hidden state and all encoder hidden states
            #receive output tensor (predictions) and new hidden state
            output1, hidden1 = self.decoder1(input1, hidden1, encoder_outputs)
            
            #place predictions in a tensor holding predictions for each token
            outputs1[t] = output1
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output1.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input1 = trg1[t] if teacher_force else top1
        
        for t in range(1, trg_len2):
            #insert input token embedding, previous hidden state and all encoder hidden states
            #receive output tensor (predictions) and new hidden state
            output2, hidden2 = self.decoder2(input2, hidden2, encoder_outputs)
            
            #place predictions in a tensor holding predictions for each token
            outputs2[t] = output2
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output2.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input1 = trg2[t] if teacher_force else top1
        
        return outputs1, outputs2

Model initialization and parameters <br>
Model diagram <br>
Weight initialization <br>
Optimizer: Adam Optimizer <br>
Criterion: Cross Entropy <br>

In [0]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM1 = len(TRG1.vocab)
OUTPUT_DIM2 = len(TRG2.vocab)
ENC_EMB_DIM = 50
DEC_EMB_DIM = 50
ENC_HID_DIM = 50
DEC_HID_DIM = 50
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec1 = Decoder(OUTPUT_DIM1, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn, weight_decoder1)
dec2 = Decoder(OUTPUT_DIM2, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn, weight_decoder2)

model = Seq2Seq(enc, dec1, dec2, device).to(device)

In [19]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(13483, 50)
    (rnn): GRU(50, 50, bidirectional=True)
    (fc): Linear(in_features=100, out_features=50, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder1): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=150, out_features=50, bias=True)
      (v): Linear(in_features=50, out_features=1, bias=False)
    )
    (embedding): Embedding(19553, 50)
    (rnn): GRU(150, 50)
    (fc_out): Linear(in_features=200, out_features=19553, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder2): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=150, out_features=50, bias=True)
      (v): Linear(in_features=50, out_features=1, bias=False)
    )
    (embedding): Embedding(17852, 50)
    (rnn): GRU(150, 50)
    (fc_out): Linear(in_features=200, out_features=17852, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [0]:
optimizer = optim.Adam(model.parameters())
TRG_PAD_IDX = TRG1.vocab.stoi[TRG1.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

The main idea behind this parallel model is derived from the loss propagation. <br>
In this case, we can have the loss being propagated from the network via two parallel paths, and hence we need to provide an additonal parameter saying retain_graph as true for the first loss we propagate. <br>

In [0]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss1 = 0
    epoch_loss2 = 0
    
    for i, batch in enumerate(iterator):
        loss1 = 0
        loss2 = 0
        src = batch.English
        trg1 = batch.Spanish
        trg2 = batch.French
        src = torch.transpose(src,0,1)
        trg1 = torch.transpose(trg1,0,1)
        trg2 = torch.transpose(trg2,0,1)
        optimizer.zero_grad()
        
        output1, output2 = model(src, trg1, trg2)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim1 = output1.shape[-1]
        output_dim2 = output2.shape[-1]
        
        
        output1 = output1[1:].view(-1, output_dim1)
        output2 = output2[1:].view(-1, output_dim2)

        #trg1 = torch.transpose(trg1,0,1)
        #trg2 = torch.transpose(trg2,0,1)
        trg1 = torch.reshape(trg1[1:], (-1,))
        trg2 = torch.reshape(trg2[1:], (-1,))
        
        #trg1 = trg1[1:].view(-1)
        #trg2 = trg2[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss1 = criterion(output1, trg1)
        loss2 = criterion(output2, trg2)
        
        loss1.backward(retain_graph=True)
        loss2.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss1 += float(loss1)
        epoch_loss2 += float(loss2)
        
    return epoch_loss1 / len(iterator), epoch_loss2 / len(iterator)

In [0]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Training loop

In [25]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss1, train_loss2 = train(model, train_iterator, optimizer, criterion, CLIP)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    #if valid_loss < best_valid_loss:
        #best_valid_loss = valid_loss
        #torch.save(model.state_dict(), 'tut6-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss1: {train_loss1:.3f} | Train PPL: {math.exp(train_loss1):7.3f}')
    print(f'\tTrain Loss2: {train_loss2:.3f} | Train PPL: {math.exp(train_loss2):7.3f}')
    torch.save(model.state_dict(), 'Seq2Seq-model-embedded.pt')

Streaming output truncated to the last 5000 lines.
Batch number:  1010
Batch number:  1011
Batch number:  1012
Batch number:  1013
Batch number:  1014
Batch number:  1015
Batch number:  1016
Batch number:  1017
Batch number:  1018
Batch number:  1019
Batch number:  1020
Batch number:  1021
Batch number:  1022
Batch number:  1023
Batch number:  1024
Batch number:  1025
Batch number:  1026
Batch number:  1027
Batch number:  1028
Batch number:  1029
Batch number:  1030
Batch number:  1031
Batch number:  1032
Batch number:  1033
Batch number:  1034
Batch number:  1035
Batch number:  1036
Batch number:  1037
Batch number:  1038
Batch number:  1039
Batch number:  1040
Batch number:  1041
Batch number:  1042
Batch number:  1043
Batch number:  1044
Batch number:  1045
Batch number:  1046
Batch number:  1047
Batch number:  1048
Batch number:  1049
Batch number:  1050
Batch number:  1051
Batch number:  1052
Batch number:  1053
Batch number:  1054
Batch number:  1055
Batch number:  1056
Batch num

In [20]:
model.load_state_dict(torch.load('Seq2Seq-model-embedded.pt'))

<All keys matched successfully>

The inference model, for translating the sentences. <br>
This is similar to the seq2seq module except the back propagation

In [0]:
def translate_sentence(sentence, src_field, trg_field1, trg_field2, model, device, max_len = 400):
    
    model.eval()
        
    tokens = [token.lower() for token in sentence]

    tokens = [src_field.init_token] + tokens + [src_field.eos_token]
        
    src_indexes = [src_field.vocab.stoi[token] for token in tokens]
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)
    src_tensor = torch.transpose(src_tensor,0,1)
    enc_src, enc_hidden = model.encoder(src_tensor)

    trg_indexes1 = [trg_field1.vocab.stoi[trg_field1.init_token]]
    trg_indexes2 = [trg_field2.vocab.stoi[trg_field2.init_token]]

    trg_indices1 = []
    trg_indices2 = []

    hidden1 = enc_hidden
    hidden2 = enc_hidden

    for i in range(max_len):
        #print(enc_src.shape)
        #print(trg_indexes1)
        trg_tensor1 = torch.LongTensor(trg_indexes1).to(device)
        #print("Print: ", trg_tensor1.shape)
        output1, hidden1 = model.decoder1(trg_tensor1, hidden1, enc_src)
        #print(output1.argmax(1)[0].item())
        pred_token1 = output1.argmax(1)[0].item()

        trg_indexes1 = [pred_token1]

        if pred_token1 == trg_field1.vocab.stoi[trg_field1.eos_token]:
            break
        
        trg_indices1.append(pred_token1)
    
    trg_tokens1 = [trg_field1.vocab.itos[i] for i in trg_indices1]

    for i in range(max_len):
        trg_tensor2 = torch.LongTensor(trg_indexes2).to(device)
        
        output2, hidden2 = model.decoder2(trg_tensor2, hidden2, enc_src)
        
        pred_token2 = output2.argmax(1)[0].item()

        trg_indexes2 = [pred_token2]

        if pred_token2 == trg_field2.vocab.stoi[trg_field2.eos_token]:
            break
        
        trg_indices2.append(pred_token2)
    
    trg_tokens2 = [trg_field2.vocab.itos[i] for i in trg_indices2]
    
    return trg_tokens1[1:], trg_tokens2[1:]

Checking the output for a random index

In [26]:
example_idx = 45

src = vars(tst.examples[example_idx])['English']
trg1 = vars(tst.examples[example_idx])['Spanish']
trg2 = vars(tst.examples[example_idx])['French']
s =" "

print(f'src = {s.join(src)}')
print(f'trg1 = {s.join(trg1)}')
print(f'trg2 = {s.join(trg2)}')

src = last friday , commissioner barnier organised a meeting in brussels with the national regulators precisely to find out what we know about the action of some of these speculators against sovereign debt .
trg1 = el viernes pasado , el comisario barnier organizó una reunión en bruselas con los entes reguladores nacionales precisamente para averiguar lo que sabemos de la acción de algunos de estos especuladores contra la deuda pública .
trg2 = vendredi dernier , le commissaire barnier a organisé une réunion à bruxelles avec les régulateurs nationaux , précisément afin de déterminer ce que nous savons de l' action de certains de ces spéculateurs contre les dettes souveraines .


In [27]:
translation1, translation2 = translate_sentence(src, SRC, TRG1, TRG2, model, device)
s = " "
print(f'predicted trg1 = {s.join(translation1)}')
print(f'predicted trg2 = {s.join(translation2)}')

predicted trg1 = , , , que un de de de de de la de de que de que que que que que que de de de de de de de de de de de de de de .
predicted trg2 = , , , , de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de . . .


Calculating the BLEU score

In [0]:
def calculate_bleu(data, src_field, trg_field1, trg_field2, model, device, max_len = 400):
    count=0
    bleu1=0
    bleu2=0
    for datum in data:
        count+=1
        src = vars(datum)['English']
        trg1 = vars(datum)['Spanish']
        trg2 = vars(datum)['French']
        
        pred_trg1, pred_trg2 = translate_sentence(src, src_field, trg_field1, trg_field2, model, device, max_len)
        
        #cut off <eos> token
        pred_trg1 = pred_trg1[:-1]
        pred_trg2 = pred_trg2[:-1]
        bleu1 += sentence_bleu([pred_trg1], trg1)
        bleu2 += sentence_bleu([pred_trg2], trg2)
        print(count)
    return bleu1/count, bleu2/count

In [0]:
bleu_score1, bleu_score2 = calculate_bleu(tst, SRC, TRG1, TRG2, model, device)
print(f'BLEU score = {bleu_score1*100:.2f}, {bleu_score2*100:.2f}')